<a href="https://colab.research.google.com/github/armanheydari/Advance-Deep-Learning_Winter-2024/blob/main/Assignment1/cmpt489_828_a1_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CMPT 489/828 Assignment 1**

Follow the instructions in this notebook and complete the missing code.

**NOTE: Do Not Change Any Provided Code or Given Variable Names!**

**Q3**. k-Nearest Neighbors (kNN) (**25 points**)

Comment/document your code to show your thought process (**1 points**)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

(a). Load dataset (**2 points**)

In [ ]:
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
# load iris dataset, this dataset contains 150 datapoints with 4 features
iris = datasets.load_iris()
# extract iris data into a numpy array
iris_data = iris.data
scaler = StandardScaler()
scaler.fit(iris_data)
iris_data = scaler.transform(iris_data)
# extract iris label into a numpy array, labels are integers: 0, 1, 2
iris_label = iris.target

iris_data, iris_label = shuffle(iris_data, iris_label)

print(iris_data.shape, iris_label.shape)

###############################################################################
# TODO:                                                                       #
# 1. Split the iris dataset into train (x_train, y_train) and                 #
# test (x_test, y_test) sets using a 80%/20% split                            #
# 2. Plot the first and second element of each datapoint in the               #
# train set (x_train) in a single scatter plot                                #
# Use the scatter() function from matplotlib.pyplot module                    #
# Use different colours to represent the corresponding classes from y_train   #
# for each datapoint                                                          #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
x_train =
y_train =

x_test =
y_test =

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

(150, 4) (150,)


(b). Implement kNN Classifier (**6 points**)

In [ ]:
class KNearestNeighbor:
    def __init__(self, x, y):
        """
        Initialise the kNN classifier, memorize the input data and label.

        Inputs:
        - x: A numpy array containing the dataset of shape (num_data, D),
        with num_data being the number of datapoints of dimension D
        - y: A numpy array containing the labels of shape (num_data,),
        y[i] is the label of x[i]
        """
        ########################################################################
        # TODO:                                                                #
        # Save the data x and label y                                          #
        ########################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        self.data =
        self.labels =

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    @staticmethod
    def l1_distance(a, b):
        """
        Calculate the l1 distance between two numpy arrays a and b.
        Inputs:
        - a: A numpy array
        - b: A numpy array

        Returns:
        -  : The L1 distance between a and b
        """
        ########################################################################
        # TODO:                                                                #
        # Calculate and return the l1 distance                                 #
        ########################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        dist_l1 =

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return dist_l1

    @staticmethod
    def l2_distance(a, b):
        """
        Calculate the l2 distance between two numpy arrays a and b.
        Inputs:
        - a: A numpy array
        - b: A numpy array

        Returns:
        -  : The L2 distance between a and b
        """
        ########################################################################
        # TODO:                                                                #
        # Calculate and return the l2 distance                                 #
        ########################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        dist_l2 =

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return dist_l2

    @staticmethod
    def new_data_rep_l2(a, b):
        """
        Calculate a new representation of image a and b, then return the l2 distance
        Inputs:
        - a: A numpy array of an image
        - b: A numpy array of an image

        Returns:
        -  : The l2 distance between your representation of a and b
        """
        ########################################################################
        # TODO:                                                                #
        # The function below is for you to implement your data representation  #
        # of an image                                                          #
        # Come back to fill in this code when you are at step (e).iii.         #
        ########################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        def new_data_rep(i):
            """
            Create a represenation of image i
            Inputs:
            - i: A numpy array of an image

            Returns:
            -  : A numpy array containing the representaion of i
            """
            pass
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        return KNearestNeighbor.l2_distance(new_data_rep(a), new_data_rep(b))

    def predict(self, x, k, dist_type='l1'):
        """
        Given a test point x, predict its class.

        Inputs:
        - x: A numpy array containing the test datapoint.
        - k: k value for kNN, number of nearest neighbors.
        - dist_type: use to choose distance functions: l1, l2 and rep_l2.

        Returns:
        - : The class of datapoint x in integer
        """
        ########################################################################
        # TODO:                                                                #
        # Use majority vote to decide the class for x                          #
        # Accommodate different distance metrics, controlled by (dist_type)    #
        ########################################################################
        # *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        pass
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****




(c). Hyperparameters tunning (**6 points**)

A function to evaluate a set of data is given below.

This is used to tune the values of k on the **val** set and evaluate the performance on the **test** set.

In [ ]:
# Example of initializing the kNN classifier
knn = KNearestNeighbor(x_train, y_train)

def evaluate(knn_classifier, x_eval, y_eval, k, dist_type='l1', return_preds=False):
    """
    Use kNN to predict values of x_eval.
    Then use the predicted values and y_eval to report classifier performance.
    Returns the point-wise accuracy.
    The point-wise accuracy is the number of true positives over the total number of data points.

    Inputs:
        - knn_classifier: Your kNN classifier
        - x_eval: A numpy array of shape (num_eval, D) containing the evaluation dataset
          consisting of num_eval samples each of dimension D.
        - y_eval: A numpy array of shape (num_eval,) containing the evaluation dataset labels, where
              y_eval[i] is the label for x_eval[i]
        - k: k value for kNN, number of nearest neighbors.
        - dist_type: use to choose distance functions: l1, l2 and rep_l2.
        - return_preds: set to true to return a list containing the predictions.

    Returns:
        - : The point-wise accuracy in the range of [0, 1]
        - : optional: a list containing the predictions
    """
    preds = np.array([knn_classifier.predict(x, k, dist_type=dist_type) for x in x_eval])
    tp = np.sum(preds==y_eval)
    if return_preds:
      return tp/len(y_eval), preds
    return tp/len(y_eval)


Use 5-fold validation to find the best value for k and the best distance metric (among L1 or L2).

**HINT**: Plot the performance average over 5 **val** sets for each value of k.

In [ ]:
###############################################################################
# TODO:                                                                       #
# 1. Perform 5-fold validation using the provided evaluate() function         #
# 2. Compare and find the best distance metric (among L1 or L2)               #
# 3. Find the optimal k for the best distance metric you found                #
# NOTE: Do Not Choose k==1                                                    #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

Report the performance of your classifier on the **test** set using the best k and best distance metric

In [ ]:
###############################################################################
# TODO:                                                                       #
# 1. Use the provided evaluate() function with your best k and distance metric#
# 2. Report the accuracy on the test set                                      #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

(d). Image dataset (**5 points**)
Repeat the above steps for the Image dataset.

**NOTE: You are given the train/val/test set split. There is no need to do 5-fold validation here.**

In [ ]:
import torchvision


# load CIFAR-10 dataset with pytorch

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

x_train = trainset.data[:4000]
y_train = np.array(trainset.targets)[:4000]

x_val = trainset.data[4000:5000]
y_val = np.array(trainset.targets)[4000:5000]

x_test = testset.data[:500]
y_test = np.array(testset.targets)[:500]

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)


class_map = {0: 'plane', 1: 'car', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9:'truck'}
###############################################################################
# TODO:                                                                       #
# Visualise 1 sample of each class from train set (x_train)                   #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


Files already downloaded and verified
Files already downloaded and verified
(4000, 32, 32, 3) (4000,)
(1000, 32, 32, 3) (1000,)
(500, 32, 32, 3) (500,)


In [ ]:
###############################################################################
# TODO:                                                                       #
# 1. Initilise kNN classifier                                                 #
# 2. Find the best k for l2 distance and                                      #
# plot the performance average over the val set for each value of k           #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


In [ ]:
##################################################################################
# TODO:                                                                          #
# 1. Report test set performance with your best k                                #
# 2. Visualize the k-nearest neighbors for the first 5 datapoints in the test set#
# NOTE: Do Not Choose k==1                                                       #
##################################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

(e). Discussion

**i**. The L1 and L2 distances when applied on the CIFAR-10 dataset, are measuring the distances between raw pixels.

Why is pixel-distance a bad representation of image distance? (**1 point**)

// your answers here

**ii**. Look at the misclassified examples from CIFAR-10 you visualized.

Take one example and explain why do you think this sample is misclassified? (**1 points**)

// your answers here

**iii**. Instead of using raw pixels of images as an input to kNN, could you suggest another data representation of the images to be fed as an input to kNN? Does it impact the kNN performance? (**3 points**)

**HINT**: An example of data representation for an image is its color histogram.

Implement your method and report the performance using the best k you found for l2 distance in step (c) on the **test** set.

In [ ]:
###############################################################################
# TODO:                                                                       #
# 1. Go back to KNearestNeighbor (class defined above) and                    #
# implement the function new_data_rep() in section (b) above                  #
# 2. Test your representation with the best k you found for l2 distance       #
# using the evaluate() function provided above                                #
# 3. Does the new representation yield better or worse performance?           #
###############################################################################
# *****BEGIN YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****